In [17]:
#dependencies
import os
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd

In [18]:
#creating scraping path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_mac64.zip
Driver has been saved in cache [/Users/jordanwright/.wdm/drivers/chromedriver/mac64/90.0.4430.24]


In [19]:
#Runs browser fom nasa site
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [20]:
# converting html throgh beautiful soup 
html = browser.html
bsoup = bs(html, 'html.parser')

In [21]:
#Path to pull title  and summary from headline
content = bsoup.select_one('div.list_text')

In [22]:
#title 
title = content.find('div', class_ = 'content_title').get_text()
title

"Perseverance's Robotic Arm Starts Conducting Science"

In [23]:
#paragraph
paragraph = content.find('div', class_ = 'article_teaser_body').get_text()
paragraph


'NASA’s newest Mars rover is beginning to study the floor of an ancient crater that once held a lake.'

In [25]:
"""Scraping featured image from JPL Mars Space Images"""

#Run Browser and visit JPL(jet propultion laboratory) site
jpl_url = "https://jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

# Variables for browser init and cleaning data through bs
jpl_html = browser.html
jpl_soup = bs(jpl_html, "html.parser")

In [28]:
# Find the featured image url from jpl_url
featured_image_url = jpl_soup.find('div', class_ = "carousel_container").article.footer.a["data-fancybox-href"]
featured_image_url = f"https://jpl.nasa.gov/spaceimages{featured_image_url}"

# Print the url for the image
print(f"featured_image_url = {featured_image_url}")

AttributeError: 'NoneType' object has no attribute 'article'

In [29]:

"""Scraping Mars Facts webpage for planet info"""

# Create path for mars facts url and use pandas to read the url into HTML table string
mfacts_url = "https://space-facts.com/mars/"
planet_profile = pd.read_html(mfacts_url)
planet_profile

# read only mars data 
mars_table = planet_profile[0]
mars_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [30]:
"""Obtain high res images for each of Mar's hemispheres from USGS Astrogeology site"""

# Path to USGS Astrogeology site and open in browser
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(usgs_url)

# Variables for browser init and cleaning data through bs
usgs_html = browser.html
usgs_soup = bs(usgs_html, "html.parser")

In [31]:

# Create empty list for hemisphere photos
hemisphere_image_urls = []

# Create a loop to append title and image url into empty list from the images
images = usgs_soup.find_all("div", class_ = "item")

for image in images:
    # Create dictionary to house each link and title individually before appending to list
    hemisphere_info = {}
    
    # title
    hemisphere_info["title"] = image.find("h3").text
    
    # follow individual link to full size image
    img_src = image.find("a")["href"]
    img_link = f"https://astrogeology.usgs.gov{img_src}"
    browser.visit(img_link)
    img_html = browser.html
    img_soup = bs(img_html, "html.parser")
    
    # image url
    img_url = img_soup.find("div", class_ = "downloads").a["href"]
    hemisphere_info["img_url"] = img_url
    
    # append info to hemisphere_image_urls
    hemisphere_image_urls.append(hemisphere_info)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [32]:
browser.quit()